In [1]:
#import findspark
from pyspark.sql import SparkSession
import os
from pyspark.sql.functions import col, to_timestamp, count, lit

In [2]:
local_storage_path = "./data/csv_files"
os.makedirs(local_storage_path, exist_ok=True)  # Create the directory if it does not exist

In [ ]:
spark = SparkSession.builder \
    .appName("AIS Data Analysis") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

In [ ]:
pathsTest1 = [
    "./data/csv_files/aisdk-2024-03-01.csv"
] #unfertig muss hier noch pfade auf unsere neuen ändern

pathsTest2 = [
    "./data/csv_files/aisdk-2024-03-01.csv"
    "./data/csv_files/aisdk-2024-03-02.csv"
]

pathsTest3 = [
    "./data/csv_files/aisdk-2024-03-01.csv"
    "./data/csv_files/aisdk-2024-03-02.csv"
    "./data/csv_files/aisdk-2024-03-03.csv"
    "./data/csv_files/aisdk-2024-03-04.csv"
    "./data/csv_files/aisdk-2024-03-05.csv"
]

pathsTest4 = [
    "./data/csv_files/aisdk-2024-03-01.csv"
    "./data/csv_files/aisdk-2024-03-02.csv"
    "./data/csv_files/aisdk-2024-03-03.csv"
    "./data/csv_files/aisdk-2024-03-04.csv"
    "./data/csv_files/aisdk-2024-03-05.csv"
    "./data/csv_files/aisdk-2024-03-01.csv"
    "./data/csv_files/aisdk-2024-03-02.csv"
    "./data/csv_files/aisdk-2024-03-03.csv"
    "./data/csv_files/aisdk-2024-03-04.csv"
    "./data/csv_files/aisdk-2024-03-05.csv"
]

### Test 1 - One File

In [ ]:
dataframes = [spark.read.csv(path, header=True, inferSchema=True) for path in pathsTest1]

# Combine all DataFrames into a single large DataFrame
combined_df = dataframes[0]
for df in dataframes[1:]:
    combined_df = combined_df.union(df)

combined_df.count()

Results: Tasks - 46 Input - 5.3 GiB InferSchema results in additional Job due to passing over file.

### Test 2 - 2 Files

In [ ]:
dataframes = [spark.read.csv(path, header=True, inferSchema=True) for path in pathsTest2]

# Combine all DataFrames into a single large DataFrame
combined_df = dataframes[0]
for df in dataframes[1:]:
    combined_df = combined_df.union(df)

combined_df.count()

Results: Tasks - 93 Input - 10.8 GiB -> Note that the input is larger than the used files, this is because the size in the spark ui is not the filesize, but the size of the loaded object. This object needs more memory due to overhead.

### Test 3 - 5 Files

In [ ]:
dataframes = [spark.read.csv(path, header=True, inferSchema=True) for path in pathsTest3]

# Combine all DataFrames into a single large DataFrame
combined_df = dataframes[0]
for df in dataframes[1:]:
    combined_df = combined_df.union(df)

combined_df.count()

Results: Tasks - 226 Input - 26.5 GiB 

### Test 4 - 10 Files

In [ ]:
dataframes = [spark.read.csv(path, header=True, inferSchema=True) for path in pathsTest4]

# Combine all DataFrames into a single large DataFrame
combined_df = dataframes[0]
for df in dataframes[1:]:
    combined_df = combined_df.union(df)

combined_df.count()

Results: Tasks - 451 Input - 52.9 GiB